In [ ]:
# Set a range of dayobs values to search - 
day_obs_min = "2024-11-13"
day_obs_max = "2024-11-13"

# HEY NOTE: script salIndexes are reused (for different purposes) when scriptqueue is restarted
# script salindex is not unique over all periods of time

# also would like:
# identifier for the scripts to replace or supplement script salIndex so that it's unique (hash for script)
# Recording of hash for ts_config_ocs or for block itself --- this may be present in new obs_env (check manage_obs_env??)

# Checking out the scriptqueue

In [ ]:
# from https://github.com/lsst-ts/ts_xml/blob/develop/python/lsst/ts/xml/enums -- import would be better 
# Informational reference point .. 
import enum

class GeneralState(enum.IntEnum):
    """CSC summaryState constants."""

    OFFLINE = 4
    STANDBY = 5
    DISABLED = 1
    ENABLED = 2
    FAULT = 3


class ScriptProcessState(enum.IntEnum):
    """ScriptQueue script.processState event constants."""

    UNKNOWN = 0
    LOADING = 1
    CONFIGURED = 2
    RUNNING = 3
    DONE = 4
    LOADFAILED = 5
    CONFIGURE_FAILED = 6
    TERMINATED = 7
    CONFIGUREFAILED = 6  # deprecated alias for CONFIGURE_FAILED

class ScriptState(enum.IntEnum):
    """ScriptState constants."""

    UNKNOWN = 0
    UNCONFIGURED = 1
    CONFIGURED = 2
    RUNNING = 3
    PAUSED = 4
    ENDING = 5
    STOPPING = 6
    FAILING = 7
    DONE = 8
    STOPPED = 9
    FAILED = 10
    CONFIGURE_FAILED = 11

# status_dict = dict((e.name, e.value) for e in GeneralState)
# process_state_dict = dict((e.name, e.value) for e in ScriptProcessState)
# script_state_dict = dict((e.name, e.value) for e in ScriptState)

# display(f"General state enum values: {status_dict}")
# display(f"Script State enum values: {script_state_dict}")
# display(f"Script Process State enum values: {process_state_dict}")

In [ ]:
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta
import astropy.units as u

from lsst_efd_client import EfdClient

In [ ]:
# Connect to the EFD 

# efd_client = EfdClient('summit_efd') 
efd_client = EfdClient('usdf_efd')

In [ ]:
# Set a range of dayobs values to search - 
t_start = Time(f"{day_obs_min}T12:00:00", format='isot', scale='utc')
#t_start = t_start + TimeDelta(0.45, format='jd')
#t_start = Time('2024-11-06T07:19:15.397500', format='isot', scale='utc') - TimeDelta(1/24, format='jd')

t_end = Time(f"{day_obs_max}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
#t_end = t_start + TimeDelta(2/24, format='jd')
print(f"Querying the EFD from {t_start.iso} to {t_end.iso}")

In [ ]:
# Query any EFD topic for the timespan day_obs_min to day_obs_max, when you don't already know the fields
# topic = lsst.sal.ScriptQueue.command_add
# fields = await efd_client.get_fields(topic)
# fields = [f for f in fields if 'private' not in f and f != 'name' and f!= "duration"]
# dd = await efd_client.select_time_series(topic, fields, tstart, tend)
# or top 5 .. 
# dd = await efd_client.select_top_n(topic, fields, 5)

In [ ]:
def apply_enum(x, column, enumvals):
    return enumvals(x[column]).name

def convert_index_to_time(x):
    return Time(x.name)

In [ ]:
# Within this time span, we need to split queries into blocks of continuous ScriptQueue uptime
# If the ScriptQueue is restarted, the salIndexes will recycle. 
# Typically this should be on the order of once a day, but let's find the boundaries using the lsst.sal.ScriptQueue.logevent_summaryState topic
topic = 'lsst.sal.ScriptQueue.logevent_summaryState'
fields = ['salIndex', 'summaryState']
dd = await efd_client.select_time_series(topic, fields, t_start, t_end)
if len(dd) == 0:
    tstops = []
    tintervals = [[t_start, t_end]]
else:
    dd['state'] = dd.apply(apply_enum, args=['summaryState', GeneralState], axis=1)
    dd['state_time'] = dd.apply(convert_index_to_time, axis=1)

    tstops = dd.query('state == "OFFLINE"').state_time.values
    #print(t_start, [e.isot for e in tstops], t_end)
    if len(tstops) == 0:
        tintervals = [[t_start, t_end]]
    if len(tstops) > 0:
        ts = tstops[0]
        ts_next = ts + TimeDelta(1 * u.second)
        tintervals = [[t_start, ts]]    
        for ts in tstops[1:]:
            tintervals.append([ts_next, ts])
            ts_next = ts + TimeDelta(1 * u.second)
        tintervals.append([ts_next, t_end])
if len(tstops) == 0:
    print(f"Found 0 ScriptQueue OFFLINE events in the time period  {t_start} to {t_end}.")
else:
    print(f"Found {len(tstops)} ScriptQueue restarts in the time period {t_start} to {t_end}, so will query in {len(tstops)+1} chunks")

In [ ]:

# # This will find JSON BLOCKS added to the queue, but not scripts executed that did not use a JSON BLOCK
# topic = 'lsst.sal.Scheduler.logevent_blockStatus'
# fields = await efd_client.get_fields(topic)
# fields = [f for f in fields if 'private' not in f]
# json_blocks_status = await efd_client.select_time_series(topic, fields, t_start, t_end)

script_stream = []
script_status = []
for tinterval in tintervals:

    # Script will find information about how scripts are configured. 
    # This gets us more information about the script parameters 
    topic = 'lsst.sal.Script.command_configure'
    fields = ['blockId', 'config',' executionId', 'salIndex']
    fields = await efd_client.get_fields(topic)
    fields = [f for f in fields if 'private' not in f]
    # blockId is only filled for JSON BLOCK activities
    scriptconfig = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scriptconfig.rename({'salIndex': 'script_salIndex'}, axis=1, inplace=True)
    scriptconfig["configure_time"] = scriptconfig.apply(convert_index_to_time, axis=1)
    
    topic = 'lsst.sal.Script.logevent_description'
    fields = ['classname', 'description', 'salIndex']
    scriptdescription = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scriptdescription.rename({'salIndex': 'script_salIndex'}, axis=1, inplace=True)
    scriptdescription['script_time'] = scriptdescription.apply(convert_index_to_time, axis=1)

    script_stream_t = pd.merge(scriptdescription, scriptconfig, on='script_salIndex', suffixes=['_d', '_r'])
    script_stream.append(script_stream_t)
    print("#info - script-stream", [e.iso for e in tinterval], len(script_stream), len(script_stream_t))

    # To find the status of each of these scripts (this is status at individual stages)
    topic = 'lsst.sal.ScriptQueue.logevent_script'
    fields = await efd_client.get_fields(topic)
    fields = ['blockId', 'path', 'processState', 'scriptState', 'salIndex', 'scriptSalIndex', 
             'timestampProcessStart', 'timestampConfigureStart', 'timestampConfigureEnd', 'timestampRunStart', 'timestampProcessEnd' ]
    scripts = await efd_client.select_time_series(topic, fields, tinterval[0], tinterval[1])
    scripts.rename({'scriptSalIndex': 'script_salIndex'}, axis=1, inplace=True)

    # Consolidate stages to see end state, as well as start and end time in one line
    scripts['finalScriptState'] = scripts['scriptState']
    script_status_t = scripts.groupby('script_salIndex').agg({'path': 'first', 'salIndex': 'max', 'finalScriptState': 'max', 'scriptState': 'unique', 'processState': 'unique', 
                                                                          'timestampProcessStart': 'max', 'timestampConfigureStart': 'max', 'timestampConfigureEnd': 'max', 
                                                                          'timestampRunStart': 'max', 'timestampProcessEnd': 'max'}).sort_values(by='timestampProcessStart')
    script_status_t['ts_run_start'] = Time(script_status_t['timestampRunStart'], format='unix').iso
    script_status_t['ts_process_start'] = Time(script_status_t['timestampProcessStart'], format='unix').iso
    script_status_t['ts_process_end'] = Time(script_status_t['timestampProcessEnd'], format='unix').iso
    script_status_t['finalScriptState'] = script_status_t.apply(apply_enum, args=['finalScriptState', ScriptState], axis=1)

    # Join with script_stream to get better descriptions 
    dd = pd.merge(script_stream_t, script_status_t, left_on='script_salIndex', right_index=True, suffixes=['', '_s'])
    script_status.append(dd)
    print("#info - script-status", [e.iso for e in tinterval], len(script_status), len(dd))
    
script_stream = pd.concat(script_stream)  # this may not be very useful or interesting compared to script-status df 
script_status = pd.concat(script_status)

In [ ]:
match = ['TakeImage', 'AddBlock', 'TakeTwilightFlats', 'TakeAOSSequence', 'FocusSweep', 'TrackTarget']
query = ''.join([f'classname.str.contains("{m}") or '  for m in match]).rstrip(' or')
sub = script_stream.query(query)
#sub.classname.unique()
#display(HTML(sub.to_html()))

In [ ]:
scols = ['classname', 'description', 'config', 'script_salIndex', 'salIndex', 'blockId', 'finalScriptState', 'scriptState', 'ts_process_start', 'ts_run_start', 'ts_process_end']

# How to subselect? ignore some things? 
ignore_commands = ['SetSummaryState', 'MuteAlarms', 'Align',]
sub = script_status
for ig in ignore_commands:
    sub = sub.query('~classname.str.contains(@ig)')

# Or only match other things?
match = ['TakeImage', 'AddBlock', 'TakeTwilightFlats', 'TakeAOSSequence', 'FocusSweep', 'TrackTarget']
query = ''.join([f'classname.str.contains("{m}") or '  for m in match]).rstrip(' or')
sub = script_status.query(query)
#display(HTML(sub[scols].to_html()))

In [ ]:
#script_status.query('config.str.contains("BLOCK-T248")')[scols]

In [ ]:
# To see what actual was configured or ran, we could look at the script queue 'queue' itself .. but it's pretty opaque 
# this is probably mostly useful for seeing - at a given time, what was running now and what was running just before
# but you'll still have to trace the salindex values into the scriptSalIndex in the scriptqueue.logevent_script 
# topic = 'lsst.sal.ScriptQueue.logevent_queue'
# fields = ['ScriptQueueID', 'currentSalIndex', 'enabled', 'priority', 'running', 'length', 'pastLength', 'pastSalIndices0', 'salIndices0',  'salIndex']
# scriptqueue = await efd_client.select_time_series(topic, fields, t_start, t_end)

# # So the best way to see all of the scripts that have been configured or run is here
# topic = 'lsst.sal.ScriptQueue.logevent_script'
# fields = await efd_client.get_fields(topic)
# fields = ['blockId', 'path', 'processState', 'scriptState', 'salIndex', 'scriptSalIndex', 
#          'timestampProcessStart', 'timestampConfigureStart', 'timestampConfigureEnd', 'timestampRunStart', 'timestampProcessEnd' ]
# scripts = await efd_client.select_time_series(topic, fields, t_start, t_end)

# # Not sure if this is useful ..
# topic = 'lsst.sal.ScriptQueue.command_add'
# fields = ['config', 'descr', 'salIndex', 'path', 'block', 'startBlock']
# scriptqueue_added = await efd_client.select_time_series(topic, fields, t_start, t_end)

In [ ]:
topics = await efd_client.get_topics()
err_topics = [t for t in topics if 'err' in t]

errs = []
for topic in err_topics:
    df = await efd_client.select_time_series(topic, ['errorCode', 'errorReport'], t_start, t_end)
    if len(df) > 0:
        df['topic'] = topic
        errs += [df]
if len(errs) > 0:
    errs = pd.concat(errs).sort_index()
    errs['time'] = Time(errs.index)
    errs.set_index('time', inplace=True)
    def strip_csc(x):
        return x.topic.replace("lsst.sal", "").replace("logevent_errorCode", "").replace(".", "") + "CSC error"
    errs['component'] = errs.apply(strip_csc, axis=1)
    errs.rename({'errorCode': 'error_code', 'errorReport': 'message_text_mod', 'topic': 'origin'}, axis=1, inplace=True)
    errs['salIndex'] = 4

print(f"Found {len(errs)} error messages")

In [ ]:
import requests

params = {"is_human" : "either",
          "is_valid" : "true",
          "has_date_begin" : True,
          "min_date_begin" : t_start.to_datetime(),
          "max_date_begin" : t_end.to_datetime(),
          "order_by" : "date_begin",
          "limit": 10000, 
         }

API_ENDPOINT = "https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages"

# we try twice because that's an issue with these services
response = requests.get(API_ENDPOINT, params)
if response.status_code != 200:
    response = requests.get(API_ENDPOINT, params)

if response.status_code != 200:
    print("Narrative log service seems to be down")
    messages = []

else:
    messages = response.json()
    for m in messages:
        m['message_text_mod'] = m['message_text'].replace("\r\n", "\n").replace("\n\n", "\n").rstrip("\n")
    messages = pd.DataFrame(messages)
    def make_time(x):
        return Time(x['date_begin'], format='isot', scale='tai')
    messages['time'] = messages.apply(make_time, axis=1)
    messages.set_index('time', inplace=True)
    messages.rename({'time_lost_type': 'error_code', 'components': 'component', 'user_id': 'origin'}, axis=1, inplace=True)
    messages['salIndex'] = 0

print(f"Found {len(messages)} messages in the narrative log")

In [ ]:
# Merge narrative log messages and error messages

jcols = ['component', 'origin', 'message_text_mod', 'error_code', 'salIndex']
joint = []
if len(errs) > 0 and len(messages) > 0:
    print("Joint narrative log and error messages")
    joint = pd.concat([errs, messages]).sort_index()

elif len(errs) > 0:
    print("Error messages only; narrative log empty")
    joint = errs

elif len(messages) > 0:
    print("Narrative log only; error messages empty")
    joint = messages

def clarify_log(x):
    if x.component is None:
        return "Log"
    elif "CSC" in x.component:
        return x.component
    elif isinstance(x.component, list):
        return "Log " + " ".join(x.component)
joint['component'] =  joint.apply(clarify_log, axis=1)

# if len(joint) > 0:
#     with option_context('display.max_colwidth', None):
#         joint.style.set_properties(**{'text-align': 'left'})
#         display(HTML(joint[jcols].style.set_properties(**{'text-align': 'left'}).to_html(justify='left')))

In [ ]:
# with option_context('display.max_colwidth', None):
#     display(HTML(script_info[cols].to_html()))

In [ ]:
script_status.index = script_status['ts_process_start'].copy()
jj = joint[jcols].rename({'component': 'classname', 'origin': 'config', 'message_text_mod': 'description', 'error_code': 'finalScriptState'}, axis=1)
jj['ts_process_start'] = jj.apply(convert_index_to_time, axis=1)
jj['script_salIndex'] = 0
efd_and_messages = pd.concat([script_status, jj]).sort_index()

def highlight_salindex(s, column):
    # Colors from https://medialab.github.io/iwanthue/
    if s.salIndex == 0:     # narrative log
        return ['background-color: #cf7ddc'] * len(s)
    elif s.salIndex ==  4:  # error messages
        return ['background-color: #9cb5d5'] * len(s)
    elif s.salIndex == 1:   # simonyi queue
        return ['background-color: #b4c546'] * len(s)
    elif s.salIndex == 2:   # aux tel queue
        return ['background-color: #bab980'] * len(s)
    elif s.salIndex == 3:   # ocs queue
        return ['background-color: #b2baad'] * len(s)
    else:
        return [''] * len(s)

print(f"Combined messages {len(efd_and_messages)}")
print("Color coding by salIndex (1/2/3 scriptqueue index) + data source (narrative log or EFD logevent_errorCode messages)")
print('')

ecols = ['classname', 'description', 'config', 'script_salIndex', 'salIndex', 'finalScriptState', 'ts_process_start', 'ts_run_start', 'ts_process_end']

with option_context('display.max_colwidth', 0):
    efd_and_messages['description'] = efd_and_messages['description'].str.wrap(100)
    display(HTML(efd_and_messages[ecols].style.apply(highlight_salindex, column=['salIndex'], axis=1).set_table_styles([dict(selector='th', props=[('text-align', 'left')])]).set_properties(**{'text-align': 'left'}).to_html()))